In [1]:
from __future__ import print_function

In [2]:
import sys

In [3]:
from pyspark.sql import SparkSession

In [4]:
file = "/Users/heewonkim/pySpark/state.csv"

In [5]:

    spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

22/11/22 16:31:24 WARN Utils: Your hostname, Heewonui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.11 instead (on interface en0)
22/11/22 16:31:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/22 16:31:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: mnmcount <file>", file=sys.stderr)
        sys.exit(-1)

    spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())
    # get the M&M data set file name
    mnm_file = sys.argv[1]
    # read the file into a Spark DataFrame
    mnm_df = (spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("/Users/heewonkim/pySpark/state.csv"))
    mnm_df.show(n=5, truncate=False)

    # aggregate count of all colors and groupBy state and color
    # orderBy descending order
    count_mnm_df = (mnm_df.select("State", "Color", "Count")
                    .groupBy("State", "Color")
                    .sum("Count")
                    .orderBy("sum(Count)", ascending=False))

    # show all the resulting aggregation for all the dates and colors
    count_mnm_df.show(n=60, truncate=False)
    print("Total Rows = %d" % (count_mnm_df.count()))

    # find the aggregate count for California by filtering
    ca_count_mnm_df = (mnm_df.select("*")
                       .where(mnm_df.State == 'CA')
                       .groupBy("State", "Color")
                       .sum("Count")
                       .orderBy("sum(Count)", ascending=False))

    # show the resulting aggregation for California
    ca_count_mnm_df.show(n=10, truncate=False)

Usage: mnmcount <file>


SystemExit: -1

/Users/heewonkim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
mnm_df = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Users/heewonkim/pySpark/mnm_dataset.csv"))
mnm_df.show(n=5, truncate=False)


+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
+-----+------+-----+
only showing top 5 rows



In [14]:
count_mnm_df = (mnm_df.select("State", "Color", "Count")
                .groupBy("State", "Color")
                .sum("Count")
                .orderBy("sum(Count)"))
count_mnm_df.show(n=60, truncate=False)

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|WY   |Brown |86110     |
|WY   |Yellow|87800     |
|WY   |Orange|87956     |
|OR   |Yellow|88129     |
|UT   |Green |88392     |
|TX   |Blue  |88466     |
|UT   |Brown |88973     |
|CA   |Blue  |89123     |
|OR   |Brown |89136     |
|UT   |Yellow|89264     |
|NV   |Red   |89346     |
|CO   |Red   |89465     |
|OR   |Green |89578     |
|WA   |Blue  |89886     |
|AZ   |Blue  |89971     |
|UT   |Blue  |89977     |
|NV   |Blue  |90003     |
|AZ   |Red   |90042     |
|NM   |Blue  |90150     |
|OR   |Red   |90286     |
|CA   |Orange|90311     |
|OR   |Blue  |90526     |
|TX   |Brown |90736     |
|AZ   |Yellow|90946     |
|CO   |Orange|90971     |
|UT   |Red   |90995     |
|WY   |Blue  |91002     |
|NM   |Green |91160     |
|NM   |Orange|91251     |
|NV   |Green |91331     |
|UT   |Orange|91341     |
|NV   |Yellow|91390     |
|WA   |Orange|91521     |
|CA   |Red   |91527     |
|AZ   |Orange|91684     |
|WY   |Red  

In [16]:
count_mnm_df.write.format("csv").mode("append").save("/Users/heewonkim/pySpark")

In [17]:
from pyspark.ml import image

In [18]:
image_dir = "/Users/heewonkim/Desktop/python_data/number_one.png"

In [20]:
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [24]:
images_df.select("image.height","image.width","image.nChannels","image.mode").show(5)

+------+-----+---------+----+
|height|width|nChannels|mode|
+------+-----+---------+----+
|   145|  200|        3|  16|
+------+-----+---------+----+

